In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from os import listdir
from glob import glob
from PIL import Image
import os
import cv2
import scipy.ndimage as ndimage
from skimage import io

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,precision_score,recall_score
from sklearn.metrics import roc_curve,auc

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import load_img, img_to_array
from keras.losses import binary_crossentropy
from keras.models import Sequential,load_model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization 
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import backend as K
from keras import activations
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers


from numpy import expand_dims


In [3]:
def load_data(files):
    X = []
    y = []
    for file in files:
        img = load_img(file, target_size = (50,50))
        pix = img_to_array(img)
        pix /= 255
        X.append(pix)
        if(file[-5] == '1'):
             y.append(1)
        elif(file[-5] == '0'):
            y.append(0)
    return np.stack(X), y

In [4]:
model=load_model("../models/final_imbalance_best_sgd2.hdf5")

In [5]:
test_df=pd.read_csv("../data/processed/testpath_df2.csv")
X_test,y_test=load_data(test_df.path)
y_test=np.vstack(y_test)
X_test.shape,y_test.shape

((44418, 50, 50, 3), (44418, 1))

In [6]:
Y_pred = model.predict(X_test)
y_pred = (Y_pred > 0.5).astype(np.int64)

1389/1389 [==============================] - 11s 8ms/step


In [8]:
def original_breast_tissue_withoutannotation(patient_id):
    patient_df = test_df.loc[test_df['P_id']== patient_id,:]
    
    max_coord = np.max((*patient_df.X,*patient_df.Y))
    base = 255*np.ones(shape = (max_coord + 50, max_coord + 50, 3)).astype(np.uint8)
    mask = 255*np.ones(shape = (max_coord + 50, max_coord + 50, 3)).astype(np.uint8)
    
    for X,Y,Class,path in zip(patient_df['X'],patient_df['Y'],patient_df['Class'],patient_df['path']):
        try:
            img = io.imread(path)
            base[Y:Y+50,X:X+50] = img
            
            
        except: pass
    
    return base

In [9]:
img_test = original_breast_tissue_withoutannotation(13019)
saveimage=Image.fromarray(img_test)

In [10]:
saveimage.save("../data/testimages/test_patient_image.png")
